# Bayesian Networks for Inference
## Group: ALT

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from pgmpy.factors.discrete import TabularCPD
from pgmpy.models import BayesianModel

## H11.2: Software

In [3]:
sprinkler_model = BayesianModel([('Cloudy', 'Sprinkler'),
                              ('Cloudy', 'Rain'),
                              ('Rain', 'WetGrass'),
                              ('Sprinkler', 'WetGrass')])



Cardinality: Number of possible values a node can assume?
CPD: conditional probability distribution

In [4]:
cpd_cloudy = TabularCPD(variable='Cloudy', variable_card=2,
                      values=[[0.5], [0.5]])
cpd_sprinkler = TabularCPD(variable='Sprinkler', variable_card=2,
                       values=[[0.5,0.9], [0.5,0.1]],
                        evidence=['Cloudy'],
                        evidence_card=[2])
cpd_rain = TabularCPD(variable='Rain', variable_card=2,
                        values=[[0.8,0.2],
                                [0.2,0.8]],
                        evidence=['Cloudy'],
                        evidence_card=[2])
cpd_grass = TabularCPD(variable='WetGrass', variable_card=2,
                      values=[[1.0,0.1,0.1, 0.01], [0., 0.9,0.9,0.99]],
                      evidence=['Rain','Sprinkler'], evidence_card=[2,2])

In [5]:
# Associating the parameters with the model structure.
sprinkler_model.add_cpds(cpd_cloudy, cpd_sprinkler, cpd_rain, cpd_grass)

# Checking if the cpds are valid for the model.
sprinkler_model.check_model()

True

In [6]:
# Get all model implied independence conditions
sprinkler_model.get_independencies()

(Cloudy _|_ WetGrass | Rain, Sprinkler)
(Sprinkler _|_ Rain | Cloudy)
(Rain _|_ Sprinkler | Cloudy)
(WetGrass _|_ Cloudy | Rain, Sprinkler)

In [7]:
print(sprinkler_model.get_cpds('Cloudy'),'\n',sprinkler_model.get_cpds('Sprinkler'),'\n',sprinkler_model.get_cpds('Rain'),'\n',sprinkler_model.get_cpds('WetGrass'))

+-----------+-----+
| Cloudy(0) | 0.5 |
+-----------+-----+
| Cloudy(1) | 0.5 |
+-----------+-----+ 
 +--------------+-----------+-----------+
| Cloudy       | Cloudy(0) | Cloudy(1) |
+--------------+-----------+-----------+
| Sprinkler(0) | 0.5       | 0.9       |
+--------------+-----------+-----------+
| Sprinkler(1) | 0.5       | 0.1       |
+--------------+-----------+-----------+ 
 +---------+-----------+-----------+
| Cloudy  | Cloudy(0) | Cloudy(1) |
+---------+-----------+-----------+
| Rain(0) | 0.8       | 0.2       |
+---------+-----------+-----------+
| Rain(1) | 0.2       | 0.8       |
+---------+-----------+-----------+ 
 +-------------+--------------+--------------+--------------+--------------+
| Rain        | Rain(0)      | Rain(0)      | Rain(1)      | Rain(1)      |
+-------------+--------------+--------------+--------------+--------------+
| Sprinkler   | Sprinkler(0) | Sprinkler(1) | Sprinkler(0) | Sprinkler(1) |
+-------------+--------------+--------------+------

In [8]:
from pgmpy.inference import VariableElimination
sprinkler_infer = VariableElimination(sprinkler_model)

In [9]:
# Computing the probability of active sprinkler given WetGrass=True.
sw = sprinkler_infer.query(variables=['Sprinkler'], evidence={'WetGrass': 1})
print(sw)

Eliminating: Cloudy: 100%|██████████| 2/2 [00:00<00:00, 1186.51it/s]

+--------------+------------------+
| Sprinkler    |   phi(Sprinkler) |
+==============+==================+
| Sprinkler(0) |           0.5702 |
+--------------+------------------+
| Sprinkler(1) |           0.4298 |
+--------------+------------------+


P( Sprinkler = True | WetGrass = True ) = 0.4298

In [10]:
swr = sprinkler_infer.query(variables=['Sprinkler'], evidence={'WetGrass': 1,'Rain':1})
print(swr)

Eliminating: Cloudy: 100%|██████████| 1/1 [00:00<00:00, 531.33it/s]

+--------------+------------------+
| Sprinkler    |   phi(Sprinkler) |
+==============+==================+
| Sprinkler(0) |           0.8055 |
+--------------+------------------+
| Sprinkler(1) |           0.1945 |
+--------------+------------------+


P( Sprinkler = True | WetGrass = True, Rain = True ) = 0.1945

## H11.3: Construction of a DAG

In [11]:
dag_model = BayesianModel([('B', 'A'),
                              ('E', 'A'),
                              ('E', 'R')])

In [12]:
cpd_b = TabularCPD(variable='B', variable_card=2,
                      values=[[0.99], [0.01]])
cpd_e = TabularCPD(variable='E', variable_card=2,
                       values=[[1-1e-6],[1e-6]])
cpd_r = TabularCPD(variable='R', variable_card=2,
                        values=[[1,0],
                                [0,1]],
                        evidence=['E'],
                        evidence_card=[2])
cpd_a = TabularCPD(variable='A', variable_card=2,
                      values=[[0.999,0.59,0.05, 0.02], [0.001, 0.41,0.95,0.98]],
                      evidence=['B','E'], evidence_card=[2,2])

In [13]:
dag_model.add_cpds(cpd_b, cpd_e, cpd_r, cpd_a)
dag_model.check_model()

True

In [14]:
print(dag_model.get_cpds('B'),'\n',dag_model.get_cpds('E'),'\n',dag_model.get_cpds('R'),'\n',dag_model.get_cpds('A'))

+------+------+
| B(0) | 0.99 |
+------+------+
| B(1) | 0.01 |
+------+------+ 
 +------+----------+
| E(0) | 0.999999 |
+------+----------+
| E(1) | 1e-06    |
+------+----------+ 
 +------+------+------+
| E    | E(0) | E(1) |
+------+------+------+
| R(0) | 1.0  | 0.0  |
+------+------+------+
| R(1) | 0.0  | 1.0  |
+------+------+------+ 
 +------+-------+------+------+------+
| B    | B(0)  | B(0) | B(1) | B(1) |
+------+-------+------+------+------+
| E    | E(0)  | E(1) | E(0) | E(1) |
+------+-------+------+------+------+
| A(0) | 0.999 | 0.59 | 0.05 | 0.02 |
+------+-------+------+------+------+
| A(1) | 0.001 | 0.41 | 0.95 | 0.98 |
+------+-------+------+------+------+


In [15]:
dag_infer = VariableElimination(dag_model)
pA = dag_infer.query(variables=['A'])
print(pA)

Eliminating: B: 100%|██████████| 3/3 [00:00<00:00, 384.43it/s]

+------+----------+
| A    |   phi(A) |
+======+==========+
| A(0) |   0.9895 |
+------+----------+
| A(1) |   0.0105 |
+------+----------+


P (A = True) = 0.0105

In [16]:
pAR = dag_infer.query(variables=['A'], evidence={'R': 1})
print(pAR)

Eliminating: E: 100%|██████████| 2/2 [00:00<00:00, 513.91it/s]

+------+----------+
| A    |   phi(A) |
+======+==========+
| A(0) |   0.5843 |
+------+----------+
| A(1) |   0.4157 |
+------+----------+


P (A | R = True) = 0.4157

In [17]:
pBA = dag_infer.query(variables=['B'], evidence={'A': 1})
print(pBA)

Eliminating: E: 100%|██████████| 2/2 [00:00<00:00, 852.93it/s]

+------+----------+
| B    |   phi(B) |
+======+==========+
| B(0) |   0.0944 |
+------+----------+
| B(1) |   0.9056 |
+------+----------+


P (B = True | A = True) = 0.9056

In [18]:
pBAR = dag_infer.query(variables=['B'], evidence={'A': 1, 'R': 1})
print(pBAR)

Eliminating: E: 100%|██████████| 1/1 [00:00<00:00, 734.94it/s]

+------+----------+
| B    |   phi(B) |
+======+==========+
| B(0) |   0.9764 |
+------+----------+
| B(1) |   0.0236 |
+------+----------+


P (B = True | A = True, R = True) = 0.0236